In [0]:
!pip install transformers
!pip install pyserini==0.8.1.0

     |████████████████████████████████| 573kB 1.4MB/s 
     |████████████████████████████████| 870kB 59.4MB/s 
     |████████████████████████████████| 1.0MB 64.1MB/s 
     |████████████████████████████████| 3.7MB 70.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=a2c2861ae8d8f927e2122a11690601a157d93610df5ca359bed8327db30ee50c
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 57.7MB 57kB/s 
     |████████████████████████████████| 1.1MB 74.5MB/s 


In [0]:
from tqdm import tqdm
import numpy as np
import random
import torch
import json
import os
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, BertConfig

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
from pyserini.search import pysearch

# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [0]:
# Download FiQA data and models
!wget https://www.dropbox.com/s/tm9zqqmxnwx1esm/finbert-qa.zip?dl=1
!unzip finbert-qa.zip?dl=1

In [0]:
config = {'max_seq_len': 512,
          'batch_size': 16,
          'learning_rate': 3e-6,
          'weight_decay': 0.01,
          'n_epochs': 2,
          'num_warmup_steps': 10000}

In [0]:
import pickle

def load_pickle(path):
    """Load pickle file.
    ----------
    Arguments:
        path: str file path
    """
    with open(path, 'rb') as f:
        return pickle.load(f)

In [0]:
# Dictionary mapping docid and qid to raw text
docid_to_text = load_pickle('finbert-qa/data/docid_to_text.pickle')
qid_to_text = load_pickle('finbert-qa/data/qid_to_text.pickle')

# List of lists:
# Each element is a list contraining [qid, list of pos docid, list of candidate docid]
train_set = load_pickle('finbert-qa/data/train_set_50.pickle')
valid_set = load_pickle('finbert-qa/data/valid_set_50.pickle')
test_set = load_pickle('finbert-qa/data/test_set_50.pickle')

# Labels
labels = load_pickle('finbert-qa/data/labels.pickle')

print("Number of questions in the training set: {}".format(len(train_set)))
print("Number of questions in the validation set: {}".format(len(valid_set)))
print("Number of questions in the test set: {}".format(len(test_set)))

Number of questions in the training set: 5676
Number of questions in the validation set: 631
Number of questions in the test set: 333


In [0]:
# Load the BERT tokenizer.
print('\nLoading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


## **Prepare data**

In [0]:
def get_input_data(dataset, max_seq_len):
    """Creates input parameters for training and validation.

    Returns:
        input_ids: List of lists
                Each element contains a list of padded/truncated numericalized
                tokens of the sequences including [CLS] and [SEP] tokens
                e.g. [[101, 2054, 2003, 102, 2449, 1029, 102], ...]
        token_type_ids: List of lists
                Each element contains a list of segment token indices to
                indicate first (question) and second (answer) parts of the inputs.
                0 corresponds to a question token, 1 corresponds an answer token
                e.g. [[0, 0, 0, 0, 1, 1, 1], ...]
        att_masks: List of lists
                Each element contains a list of mask values to avoid
                performing attention on padding token indices.
                1 for tokens that are NOT MASKED, 0 for MASKED tokens.
                e.g. [[1, 1, 1, 1, 1, 1, 1], ...]
        labels: List of 1's and 0's incidating relevacy of answer
    -----------------
    Arguements:
        dataset: List of lists in the form of [qid, [pos ans], [ans cands]]
    """
    input_ids = []
    token_type_ids = []
    att_masks = []
    labels = []

    for i, seq in enumerate(tqdm(dataset)):
        qid, ans_labels, cands = seq[0], seq[1], seq[2]
        # Map question id to text
        q_text = qid_to_text[qid]
        # For each answer in the candidates
        for docid in cands:
            # Map the docid to text
            ans_text = docid_to_text[docid]
            # Encode the sequence using BERT tokenizer
            encoded_seq = tokenizer.encode_plus(q_text, ans_text,
                                                max_length=max_seq_len,
                                                pad_to_max_length=True,
                                                return_token_type_ids=True,
                                                return_attention_mask = True)
            # Get parameters
            input_id = encoded_seq['input_ids']
            token_type_id = encoded_seq['token_type_ids']
            att_mask = encoded_seq['attention_mask']

            # If an answer is in the list of relevant answers assign
            # positive label
            if docid in ans_labels:
                label = 1
            else:
                label = 0

            # Each parameter list has the length of the max_seq_len
            assert len(input_id) == max_seq_len, "Input id dimension incorrect!"
            assert len(token_type_id) == max_seq_len, "Token type id dimension incorrect!"
            assert len(att_mask) == max_seq_len, "Attention mask dimension incorrect!"

            input_ids.append(input_id)
            token_type_ids.append(token_type_id)
            att_masks.append(att_mask)
            labels.append(label)

    return input_ids, token_type_ids, att_masks, labels

In [0]:
def get_dataloader(dataset, type, max_seq_len, batch_size):
    """Creates train and validation DataLoaders with input_ids,
    token_type_ids, att_masks, and labels

    Returns:
        train_dataloader: DataLoader object
        validation_dataloader: DataLoader object

    -----------------
    Arguements:
        dataset: List of lists in the form of [qid, [pos ans], [ans cands]]
        type: str - 'train' or 'validation'
        max_seq_len: int
        batch_size: int
    """
    input_id, token_type_id, \
    att_mask, label = get_input_data(dataset, max_seq_len)

    # Convert all inputs to torch tensors
    input_ids = torch.tensor(input_id)
    token_type_ids = torch.tensor(token_type_id)
    att_masks = torch.tensor(att_mask)
    labels = torch.tensor(label)

    # Create the DataLoader for our training set.
    data = TensorDataset(input_ids, token_type_ids, att_masks, labels)
    if type == "train":
        sampler = RandomSampler(data)
    else:
        sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    
    return dataloader

In [0]:
# Get dataloaders
train_dataloader = get_dataloader(train_set, 'train', 
                                  config['max_seq_len'], 
                                  config['batch_size'])
validation_dataloader = get_dataloader(valid_set, 'validation', 
                                       config['max_seq_len'], 
                                       config['batch_size'])

print("\n\nSize of the training DataLoader: {}".format(len(train_dataloader)))
print("Size of the validation DataLoader: {}".format(len(validation_dataloader)))

100%|██████████| 631/631 [02:15<00:00,  4.67it/s]




Size of the training DataLoader: 17738
Size of the validation DataLoader: 1972


## **Model**

In [0]:
# Load BertForSequenceClassification - pretrained BERT model 
# with a single linear classification layer on top

model_path = "finbert-qa/model/bert-qa"
model = BertForSequenceClassification.from_pretrained(model_path, cache_dir=None, num_labels=2)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## **Training/Validation methods**

In [0]:
def get_accuracy(preds, labels):
    """Compute the accuracy of binary predictions.

    Returns:
        accuracy: float
    -----------------
    Arguments:
        preds: Numpy list with two columns of probabilities for each label
        labels: List of labels
    """
    # Get the label (column) with the higher probability
    predictions = np.argmax(preds, axis=1).flatten()
    labels = labels.flatten()
    # Compute accuracy
    accuracy = np.sum(predictions == labels) / len(labels)

    return accuracy

In [0]:
def train(model, train_dataloader, optimizer, scheduler):
    """Trains the model and returns the average loss and accuracy.

    Returns:
        avg_loss: Float
        avg_acc: Float
    ----------
    Arguements:
        model: Torch model
        train_dataloader: DataLoader object
        optimizer: Optimizer object
        scheduler: Scheduler object
    """
    # Cumulated Training loss and accuracy
    total_loss = 0
    train_accuracy = 0
    # Track the number of steps
    num_steps = 0
    # Set model in train mode
    model.train()
    # For each batch of training data
    for step, batch in enumerate(tqdm(train_dataloader)):
        # Get tensors and move to gpu
        # batch contains four PyTorch tensors:
        #   [0]: input ids
        #   [1]: token_type_ids
        #   [2]: attention masks
        #   [3]: labels
        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Zero the gradients
        model.zero_grad()
        # Forward pass: the model will return the loss and the logits
        outputs = model(b_input_ids,
                        token_type_ids = b_token_type_ids,
                        attention_mask = b_input_mask,
                        labels = b_labels)

        # Get loss and predictions
        loss = outputs[0]
        logits = outputs[1]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for a batch
        tmp_accuracy = get_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        train_accuracy += tmp_accuracy

        # Track the number of batches
        num_steps += 1

        # Accumulate the training loss over all of the batches
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update scheduler
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_loss = total_loss / len(train_dataloader)
    avg_acc = train_accuracy/num_steps

    return avg_loss, avg_acc

In [0]:
def validate(model, validation_dataloader):
    """Validates the model and returns the average loss and accuracy.

    Returns:
        avg_loss: Float
        avg_acc: Float
    ----------
    Arguements:
        model: Torch model
        validation_dataloader: DataLoader object
    """
    # Set model to evaluation mode
    model.eval()
    # Cumulated Training loss and accuracy
    total_loss = 0
    eval_accuracy = 0
    # Track the number of steps
    num_steps = 0

    # For each batch of the validation data
    for batch in tqdm(validation_dataloader):
        # Move tensors from batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from the dataloader
        b_input_ids, b_token_type_ids, b_input_masks, b_labels = batch
        # Don't to compute or store gradients
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids = b_token_type_ids,
                            attention_mask = b_input_masks,
                            labels= b_labels)
        # Get loss and logits
        loss = outputs[0]
        logits = outputs[1]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = get_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of steps
        num_steps += 1

        total_loss += loss.item()

    # Calculate loss and accuracy
    avg_loss = total_loss / len(validation_dataloader)
    avg_acc = eval_accuracy/num_steps

    return avg_loss, avg_acc

## **Fine-tune FinBERT-QA**

In [0]:
optimizer = AdamW(model.parameters(), 
                  lr = config['learning_rate'], 
                  weight_decay = config['weight_decay'])

n_epochs = config['n_epochs']


# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * n_epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = config['num_warmup_steps'],
                                            num_training_steps = total_steps)

In [0]:
# Train and validate the model and print the average loss and accuracy.

# Lowest validation lost
best_valid_loss = float('inf')

for epoch in range(n_epochs):
    # Evaluate training loss
    train_loss, train_acc = train(model, train_dataloader, optimizer, scheduler)
    # Evaluate validation loss
    valid_loss, valid_acc = validate(model, validation_dataloader)
    # At each epoch, if the validation loss is the best
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'finbert-qa/model/' + \
        str(epoch+1)+ '_finbert-qa.pt')

    print("\n\n Epoch {}:".format(epoch+1))
    print("\t Train Loss: {} | Train Accuracy: {}%".format(round(train_loss, 3), round(train_acc*100, 2)))
    print("\t Validation Loss: {} | Validation Accuracy: {}%\n".format(round(valid_loss, 3), round(valid_acc*100, 2)))

  0%|          | 0/17738 [00:00<?, ?it/s]



 Epoch 1:
	 Train Loss: 0.09 | Train Accuracy: 97.8%
	 Validation Loss: 0.078 | Validation Accuracy: 98.06%



 88%|████████▊ | 15589/17738 [3:42:09<30:39,  1.17it/s]

## **Evaluate**

In [0]:
def get_rank(model, test_set, max_seq_len):
    """Re-ranks the candidates answers for each question.

    Returns:
        qid_pred_rank: Dictionary
            key - qid
            value - list of re-ranked candidates
    -------------------
    Arguments:
        model - PyTorch model
    """
    # Initiate empty dictionary
    qid_pred_rank = {}
    # Set model to evaluation mode
    model.eval()
    # For each element in the test set
    for i, seq in enumerate(tqdm(test_set)):
        # question id, list of rel answers, list of candidates
        qid, label, cands = seq[0], seq[1], seq[2]
        # Map question id to text
        q_text = qid_to_text[qid]
        # Convert list to numpy array
        cands_id = np.array(cands)
        # Empty list for the probability scores of relevancy
        scores = []

        # For each answer in the candidates
        for docid in cands:
            # Map the docid to text
            ans_text = docid_to_text[docid]
            # Create inputs for the model
            encoded_seq = tokenizer.encode_plus(q_text, ans_text,
                                            max_length=max_seq_len,
                                            pad_to_max_length=True,
                                            return_token_type_ids=True,
                                            return_attention_mask = True)

            # Numericalized, padded, clipped seq with special tokens
            input_ids = torch.tensor([encoded_seq['input_ids']]).to(device)
            # Specify question seq and answer seq
            token_type_ids = torch.tensor([encoded_seq['token_type_ids']]).to(device)
            # Sepecify which position is part of the seq which is padded
            att_mask = torch.tensor([encoded_seq['attention_mask']]).to(device)

            # Don't calculate gradients
            with torch.no_grad():
            # Forward pass, calculate logit predictions for each QA pair
                outputs = model(input_ids,
                                token_type_ids=token_type_ids,
                                attention_mask=att_mask)
            # Get the predictions
            logits = outputs[0]
            # Apply activation function
            pred = softmax(logits, dim=1)
            # Move logits and labels to CPU
            pred = pred.detach().cpu().numpy()
            # Append relevant scores to list (where label = 1)
            scores.append(pred[:,1][0])
        # Get the indices of the sorted similarity scores
        sorted_index = np.argsort(scores)[::-1]
        # Get the list of docid from the sorted indices
        ranked_ans = cands_id[sorted_index]
        # Dict - key: qid, value: ranked list of docids
        qid_pred_rank[qid] = ranked_ans

    return qid_pred_rank

In [0]:
from finbert-qa.evaluate import *

trained_model_path = "finbert-qa/model/finbert-qa/2_finbert-qa-50_512_16_3e6.pt"
# Load model
model.load_state_dict(torch.load(trained_model_path), strict=False)

# Get rank
qid_pred_rank = get_rank(model, test_set, config['max_seq_len'])

k = 10
num_q = len(test_set)

# Evaluate
MRR, average_ndcg, precision, rank_pos = evaluate(qid_pred_rank, labels, k)

print("Average nDCG@{0} for {1} queries: {2:.3f}".format(k, num_q, average_ndcg))
print("MRR@{0} for {1} queries: {2:.3f}".format(k, num_q, MRR))
print("Average Precision@1 for {0} queries: {1:.3f}".format(num_q, precision))

## **Predict**

In [0]:
def predict(model, q_text, cands, max_seq_len):
    """Re-ranks the candidates answers for each question.

    Returns:
        ranked_ans: list of re-ranked candidate docids
        sorted_scores: list of relevancy scores of the answers
    -------------------
    Arguments:
        model - PyTorch model
        q_text - str - query
        cands -List of retrieved candidate docids
    """
    model.eval()
    # Convert list to numpy array
    cands_id = np.array(cands)
    # Empty list for the probability scores of relevancy
    scores = []
    # For each answer in the candidates
    for docid in cands:
        # Map the docid to text
        ans_text = docid_to_text[docid]
        # Create inputs for the model
        encoded_seq = tokenizer.encode_plus(q_text, ans_text,
                                            max_length=max_seq_len,
                                            pad_to_max_length=True,
                                            return_token_type_ids=True,
                                            return_attention_mask = True)

        # Numericalized, padded, clipped seq with special tokens
        input_ids = torch.tensor([encoded_seq['input_ids']]).to(device)
        # Specify question seq and answer seq
        token_type_ids = torch.tensor([encoded_seq['token_type_ids']]).to(device)
        # Sepecify which position is part of the seq which is padded
        att_mask = torch.tensor([encoded_seq['attention_mask']]).to(device)
        # Don't calculate gradients
        with torch.no_grad():
            # Forward pass, calculate logit predictions for each QA pair
            outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=att_mask)
        # Get the predictions
        logits = outputs[0]
        # Apply activation function
        pred = softmax(logits, dim=1)
        # Move logits and labels to CPU
        pred = pred.detach().cpu().numpy()
        # Append relevant scores to list (where label = 1)
        scores.append(pred[:,1][0])
        # Get the indices of the sorted similarity scores
        sorted_index = np.argsort(scores)[::-1]
        # Get the list of docid from the sorted indices
        ranked_ans = list(cands_id[sorted_index])
        sorted_scores = list(np.around(sorted(scores, reverse=True),decimals=3))

    return ranked_ans, sorted_scores

def get_rel(labels, cands):
    """Get relevant positions of the hits.

    Returns: List of 0's and 1's incidating a relevant answer
    -------------------
    Arguments:
        labels: List of relevant docids
        cands: List of candidate docids
    """
    rel = []
    for cand in cands:
        if cand in labels:
            rel.append(1)
        else:
            rel.append(0)

    return rel

In [0]:
FIQA_INDEX = "fiqa/lucene-index-fiqa"

searcher = pysearch.SimpleSearcher(FIQA_INDEX)

In [0]:
seq = test_set[91]
qid, label, cands = seq[0], seq[1], seq[2]
q_text = qid_to_text[qid]
query = q_text
print(query)
# query = "Which investments are the best?"

In [0]:
hits = searcher.search(query, k=50)
self.cands = []

for i in range(0, len(hits)):
    cands.append(int(hits[i].docid))

trained_model_path = "fiqa/model/finbert-qa/2_finbert-qa-50_512_16_3e6.pt"
# Load model
model.load_state_dict(torch.load(trained_model_path), strict=False)

rank, scores = predict(model, query, cands)

In [0]:
k = 5

print("Question: \n\t{}\n".format(query))
print("Top-{} Answers: \n".format(k))
for i in range(0, 10):
    print("{}.\t{}\n".format(i+1, docid_to_text[rank[i]]))

In [0]:
cand_rel = get_rel(label, cands)
print("Retriever: \n\t Ranking: {}\n\n\t {}".format(cands[:10], cand_rel[:10]))
pred_rel = get_rel(label, rank)
print("Re-ranker: \n\t Ranking: {}\n\n\t Scores: {}\n\n\t {}".format(rank[:10], scores[:10], pred_rel[:10]))
print("Label: \n\t{}".format(label))

In [0]:
print("Question: \n\t{}\n".format(query))
print("Answer Re-ranker\n")
print("Answer: \n\t{}\n".format(docid_to_text[rank[0]]))
print("Answer Retriever\n")
print("Answer: \n\t{}\n".format(docid_to_text[cands[0]]))
print("label: \n\t{}".format(docid_to_text[label[1]]))